<a href="https://colab.research.google.com/github/ali3brt/-Auto-GPT-An-Autonomous-GPT-4-Experiment-/blob/main/create_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and store data

This notebook contains information on downloading the Quandl Wiki stock prices and a few other sources that we use throughout the book.

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

## Set Data Store path

Modify path if you would like to store the data elsewhere and change the notebooks accordingly

In [6]:
DATA_STORE = Path('content/assets.h5')

## Quandl Wiki Prices

> Quandl has been [acuqired by NASDAQ](https://www.nasdaq.com/about/press-center/nasdaq-acquires-quandl-advance-use-alternative-data) in late 2018. In 2021, NASDAQ [integrated Quandl's data platform](https://data.nasdaq.com/). Free US equity data is still available under a [new URL](https://data.nasdaq.com/databases/WIKIP/documentation), subject to the limitations mentioned below.

[NASDAQ](https://data.nasdaq.com/) makes available a [dataset](/home/stefan/drive/machine-learning-for-trading/data/create_datasets.ipynb) with stock prices, dividends and splits for 3000 US publicly-traded companies. Prior to its acquisition (April 11, 2018), Quandl announced the end of community support (updates). The historical data are useful as a first step towards demonstrating the application of the machine learning solutions, just ensure you design and test your own algorithms using current, professional-grade data.

1. Follow the instructions to create a free [NASDAQ account](https://data.nasdaq.com/sign-up)
2. [Download](https://data.nasdaq.com/tables/WIKIP/WIKI-PRICES/export) the entire WIKI/PRICES data
3. Extract the .zip file,
4. Move to this directory and rename to wiki_prices.csv
5. Run the below code to store in fast HDF format (see [Chapter 02 on Market & Fundamental Data](../02_market_and_fundamental_data) for details).

In [11]:
import os
import pandas as pd

# Define a valid path for the HDF5 file
DATA_STORE = '/content/wiki_prices.h5'

# Ensure the directory exists
os.makedirs(os.path.dirname(DATA_STORE), exist_ok=True)

# Load and process the data
df = (pd.read_csv('wiki_prices.csv',
                  parse_dates=['date'],
                  index_col=['date', 'ticker'],
                  infer_datetime_format=True)
      .sort_index())

print(df.info(show_counts=True))

# Store the data in an HDF5 file
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13630293 entries, ('1962-01-02', 'ARNC') to ('2018-03-27', 'TNK')
Data columns (total 12 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   open         13629755 non-null  float64
 1   high         13630241 non-null  float64
 2   low          13630241 non-null  float64
 3   close        13630291 non-null  float64
 4   volume       13630292 non-null  float64
 5   ex-dividend  13630292 non-null  float64
 6   split_ratio  13630291 non-null  float64
 7   adj_open     13629755 non-null  float64
 8   adj_high     13630241 non-null  float64
 9   adj_low      13630241 non-null  float64
 10  adj_close    13630291 non-null  float64
 11  adj_volume   13630292 non-null  float64
dtypes: float64(12)
memory usage: 1.3+ GB
None


### Wiki Prices Metadata

> QUANDL used to make some stock meta data be available on its website; I'm making the file available to allow readers to run some examples in the book:

Instead of using the QUANDL API, load the file `wiki_stocks.csv` as described and store in HDF5 format.

In [13]:
df = pd.read_csv('wiki_stocks.csv')

# Print DataFrame information
print(df.info(show_counts=True))

with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/stocks', df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    3199 non-null   object
 1   name    3199 non-null   object
dtypes: object(2)
memory usage: 50.1+ KB
None


## S&P 500 Prices

The following code downloads historical S&P 500 prices from FRED (only last 10 years of daily data is freely available)

In [14]:
df = web.DataReader(name='SP500', data_source='fred', start=2009).squeeze().to_frame('close')
print(df.info())
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/fred', df)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2609 entries, 2015-01-14 to 2025-01-13
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   2515 non-null   float64
dtypes: float64(1)
memory usage: 40.8 KB
None


Alternatively, download S&P500 data from [stooq.com](https://stooq.com/q/?s=%5Espx&c=1d&t=l&a=lg&b=0); at the time of writing the data was available since 1789. You can switch from Polish to English on the lower right-hand side.

We store the data from 1950-2020:

In [16]:
sp500_stooq = (pd.read_csv('^spx_d.csv', index_col=0,
                     parse_dates=True).loc['1950':'2025'].rename(columns=str.lower))
print(sp500_stooq.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18867 entries, 1950-05-01 to 2025-01-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    18867 non-null  float64
 1   high    18867 non-null  float64
 2   low     18867 non-null  float64
 3   close   18867 non-null  float64
 4   volume  18867 non-null  float64
dtypes: float64(5)
memory usage: 884.4 KB
None


In [17]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stooq', sp500_stooq)

### S&P 500 Constituents

The following code downloads the current S&P 500 constituents from [Wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies).

In [18]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]

In [19]:
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [30]:
# Correct column names
df.columns = ['ticker', 'name', 'gics_sector', 'gics_sub_industry', 'location']

# Set 'ticker' as the index
df = df.set_index('ticker')


In [31]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, Industrial Conglomerates to Pharmaceuticals
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               503 non-null    object
 1   gics_sector        503 non-null    object
 2   gics_sub_industry  503 non-null    int64 
 3   location           503 non-null    object
dtypes: int64(1), object(3)
memory usage: 19.6+ KB
None


In [32]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stocks', df)

## Metadata on US-traded companies

The following downloads several attributes for [companies](https://www.nasdaq.com/screening/companies-by-name.aspx) traded on NASDAQ, AMEX and NYSE

> Update: unfortunately, NASDAQ has disabled automatic downloads. However, you can still access and manually download the files at the below URL when you fill in the exchange names. So for AMEX, URL becomes `https://www.nasdaq.com/market-activity/stocks/screener?exchange=AMEX&letter=0&render=download`.
>

In [33]:
# no longer works!
url = 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange={}&render=download'
exchanges = ['NASDAQ', 'AMEX', 'NYSE']
df = pd.concat([pd.read_csv(url.format(ex)) for ex in exchanges]).dropna(how='all', axis=1)
df = df.rename(columns=str.lower).set_index('symbol').drop('summary quote', axis=1)
df = df[~df.index.duplicated()]
print(df.info())

KeyboardInterrupt: 

In [34]:
df.head()

,name,gics_sector,gics_sub_industry,location
ticker,,,,
Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


### Convert market cap information to numerical format

Market cap is provided as strings so we need to convert it to numerical format.

In [35]:
mcap = df[['marketcap']].dropna()
mcap['suffix'] = mcap.marketcap.str[-1]
mcap.suffix.value_counts()

KeyError: "None of [Index(['marketcap'], dtype='object')] are in the [columns]"

Keep only values with value units:

In [36]:
mcap = mcap[mcap.suffix.str.endswith(('B', 'M'))]
mcap.marketcap = pd.to_numeric(mcap.marketcap.str[1:-1])
mcaps = {'M': 1e6, 'B': 1e9}
for symbol, factor in mcaps.items():
    mcap.loc[mcap.suffix == symbol, 'marketcap'] *= factor
mcap.info()

NameError: name 'mcap' is not defined

In [37]:
df['marketcap'] = mcap.marketcap
df.marketcap.describe(percentiles=np.arange(.1, 1, .1).round(1)).apply(lambda x: f'{int(x):,d}')

NameError: name 'mcap' is not defined

### Store result

The file `us_equities_meta_data.csv` contains a version of the data used for many of the examples. Load using
```
df = pd.read_csv('us_equities_meta_data.csv')
```
and proceed to store in HDF5 format.

In [39]:
df = pd.read_csv('us_equities_meta_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6834 entries, 0 to 6833
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     6834 non-null   object 
 1   name       6834 non-null   object 
 2   lastsale   6718 non-null   float64
 3   marketcap  5766 non-null   float64
 4   ipoyear    3038 non-null   float64
 5   sector     5288 non-null   object 
 6   industry   5288 non-null   object 
dtypes: float64(3), object(4)
memory usage: 373.9+ KB


In [40]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('us_equities/stocks', df.set_index('ticker'))

## MNIST Data

In [2]:
mnist = fetch_openml('mnist_784', version=1)

NameError: name 'fetch_openml' is not defined

In [46]:
print(mnist.DESCR)

NameError: name 'mnist' is not defined

In [43]:
mnist.keys()

NameError: name 'mnist' is not defined

In [44]:
mnist_path = Path('mnist')
if not mnist_path.exists():
    mnist_path.mkdir()

In [45]:
np.save(mnist_path / 'data', mnist.data.astype(np.uint8))
np.save(mnist_path / 'labels', mnist.target.astype(np.uint8))

NameError: name 'mnist' is not defined

## Fashion MNIST Image Data

We will use the Fashion MNIST image data created by [Zalando Research](https://github.com/zalandoresearch/fashion-mnist) for some demonstrations.

In [1]:
fashion_mnist = fetch_openml(name='Fashion-MNIST')

NameError: name 'fetch_openml' is not defined

In [4]:
print(fashion_mnist.DESCR)

NameError: name 'fashion_mnist' is not defined

In [3]:
label_dict = {0: 'T-shirt/top',
              1: 'Trouser',
              2: 'Pullover',
              3: 'Dress',
              4: 'Coat',
              5: 'Sandal',
              6: 'Shirt',
              7: 'Sneaker',
              8: 'Bag',
              9: 'Ankle boot'}

In [5]:
fashion_path = Path('fashion_mnist')
if not fashion_path.exists():
    fashion_path.mkdir()

NameError: name 'Path' is not defined

In [6]:
pd.Series(label_dict).to_csv(fashion_path / 'label_dict.csv', index=False, header=None)

NameError: name 'pd' is not defined

In [7]:
np.save(fashion_path / 'data', fashion_mnist.data.astype(np.uint8))
np.save(fashion_path / 'labels', fashion_mnist.target.astype(np.uint8))


NameError: name 'np' is not defined

## Bond Price Indexes

The following code downloads several bond indexes from the Federal Reserve Economic Data service ([FRED](https://fred.stlouisfed.org/))

> Warning: Unfortunately, most of this data has been [recently removed](https://news.research.stlouisfed.org/2022/01/ice-benchmark-administration-ltd-iba-data-to-be-removed-from-fred/) from the FRED service. It is not important for the examples in the book, so you can just ignore this.

In [8]:
securities = {'BAMLCC0A0CMTRIV'   : 'US Corp Master TRI',
              'BAMLHYH0A0HYM2TRIV': 'US High Yield TRI',
              'BAMLEMCBPITRIV'    : 'Emerging Markets Corporate Plus TRI',
              'GOLDAMGBD228NLBM'  : 'Gold (London, USD)',
              'DGS10'             : '10-Year Treasury CMR',
              }

df = web.DataReader(name=list(securities.keys()), data_source='fred', start=2000)
df = df.rename(columns=securities).dropna(how='all').resample('B').mean()

with pd.HDFStore(DATA_STORE) as store:
    store.put('fred/assets', df)

NameError: name 'web' is not defined